# 목차
1. 데이터 셋 확인
* 데이터 준비 및 모듈 import 
* null check 
* Target label 확인

## 데이터 셋 확인
* 데이터 준비 및 모듈 import 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn import model_selection
plt.style.use('seaborn')
sns.set(font_scale=2.5) 
import missingno as msno

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
data = pd.read_csv("/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")

In [ ]:
data.head()

In [ ]:
data.shape

전체 1599개의 데이터와 12개의 feature(특성)로 이루어져 있습니다.

| 변수 | 설명 | 타입 |
|--------|:--------:|-------:|
| quality | 와인의 좋은 정도(target) | int |
| fixed acidity  | 고정 산도 | float |
| volatile acidity | 휘발성 산도 | float |
| citric acid | 구연산 | float |
| residual sugar | 잔류 설탕 | float |
| chlorides  | 염화물 | float |
| free sulfur dioxide | 유리 이산화황 | float |
|  sulfur  | 황 | float |
| dioxide  | 이산화물 | float |
| density  | 밀도 | float |
| pH | ph | float |
| sulphates | 황산염 | float |
|  alcohol  | 알코올 | float |

quality를 제외한 모든 특성 값은 float(실수형)입니다.

* Null data 체크

In [ ]:
for col in data.columns:
    print("column: {:>30}\tPercent of NaN value:{:.2f}%".format(col,100*(data[col].isnull().sum()/ data[col].shape[0])) )

데이터에는 null값은 없습니다.

* target label 확인
 * target data가 한쪽으로 치우쳐 있다면 정확한 예측을 할 수 없기 때문에 확인하도록 하겠습니다.

In [ ]:
f, ax = plt.subplots(1,2, figsize = (18,8))
data["quality"].value_counts().plot.pie(autopct = "%1.1f%%",ax = ax[0], shadow=True)
ax[0].set_title("Pie plot - quality")
ax[1].set_title("Count plot - quality")

sns.countplot("quality",data = data, ax = ax[1],order = data['quality'].value_counts().index)

plt.show()

데이터가 비교적 고르게 분포되어 있습니다.

## Exploratory data analysis(데이터 분석 탐구)
* 특성 별 상관관계 분석

In [ ]:
heatmap_data = data
colormap = plt.cm.RdBu
plt.figure(figsize=(14, 12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(heatmap_data.astype(float).corr(), linewidths=0.1, vmax=1.0,
           square=True, cmap=colormap, linecolor='white', annot=True, annot_kws={"size": 16})

del heatmap_data

서로 강한 상관관계를 가지는 feature들이 없다는 것을 알 수 있습니다. 이러한 것은 우리가 모델을 학습 시킬 때 불필요한 feature 가 없다는 것을 의미 합니다.

* 특성 별 box plot 

In [ ]:
plt.figure(figsize = (20,60))
for ind,col in enumerate(data.columns[:-1]):
    
    plt.subplot(6,2,ind+1)
    plt.title(col, color = "black", fontweight='bold', fontsize = 20)
    sns.boxplot('quality', col, data = data)
    plt.xlabel("")

## Feature engineering
* 실수형 변수 그룹화
  데이터를 효과적으로 학습하기 위해서 10개의 그룹으로 만들겠습니다.

In [ ]:
temp = data.copy()
for col in data.columns[:-1]:
    
    temp[col+"_group"] = 0
    temp[col+"_group"] = pd.qcut(data[col], 10, labels=np.arange(0, 10))
    temp.drop([col],axis=1,inplace = True )
temp.head()

In [ ]:
temp = data.copy()
temp["test"] = 0

bins = (2, 6.5, 8)
group_names = ['bad', 'good']
temp['quality'] = pd.cut(temp['quality'], bins = bins, labels = group_names)

In [ ]:


y = temp.quality
x = temp.drop(["quality"], axis = 1)



test_size = 0.20
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = test_size, random_state = 206)



In [ ]:
temp.head(5)

In [ ]:
ran = RandomForestClassifier(random_state=1)
knn = KNeighborsClassifier()
log = LogisticRegression()
xgb = XGBClassifier()
gbc = GradientBoostingClassifier()
svc = SVC(probability=True)
ext = ExtraTreesClassifier()
ada = AdaBoostClassifier()
gnb = GaussianNB()
gpc = GaussianProcessClassifier()
bag = BaggingClassifier()
# 리스트 준비
models = [ran, knn, log, xgb, gbc, svc, ext, ada, gnb, gpc, bag]         
model_names = ['Random Forest', 'K Nearest Neighbour', 'Logistic Regression', 'XGBoost', 'Gradient Boosting', 'SVC', 'Extra Trees', 'AdaBoost', 'Gaussian Naive Bayes', 'Gaussian Process', 'Bagging Classifier']
scores = {}

# 학습 및 교차 검증
for ind, mod in enumerate(models):
    mod.fit(X_train, Y_train)
    acc = cross_val_score(mod, X_train, Y_train, scoring = "accuracy", cv = 10)
    scores[model_names[ind]] = acc

In [ ]:
results = pd.DataFrame(scores).T
results['mean'] = results.mean(1)
result_df = results.sort_values(by='mean', ascending=False)
result_df.head(11)


result_df = result_df.drop(['mean'], axis=1)
sns.boxplot(data=result_df.T, orient='h')
plt.title('Machine Learning Algorithm Accuracy Score \n')
plt.xlabel('Accuracy Score (%)');

In [ ]:


result_df.style.background_gradient("Blues_r")



In [ ]:
result_df.max().max()